In [1]:
import pandas as pd
from gensim import models
import dask.dataframe as dd
import dask.bag as db

model = models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False, no_header=True)

In [2]:
docs = dd.read_parquet("../output.pq/")
texts = docs['News_Tokens'].compute()
bag = db.from_sequence(texts)
sentences = bag.flatten()
words = sentences.map(lambda x: x.split()).flatten().distinct().compute()

In [3]:
results = []
threshold = 0.7
skipped=0
for word in words:
    try:
        # Получаем 10 ближайших слов
        similar_words = model.most_similar(word, topn=10)
        filtered_words = [(w, sim) for w, sim in similar_words if sim >= threshold]
        results.append([word, filtered_words])
    except KeyError:
        # Если слово не в модели, пропускаем его
        print(f"Слово '{word}' не найдено в модели.")
        skipped += 1
        continue

df_results = pd.DataFrame(results, columns=["Word", "Most_Similar_Word"])
print(f"Пропустили {skipped} слов")
df_results.to_csv("sim-words_sent_5_glove.csv", index=False)

Слово 'общенародный' не найдено в модели.
Слово 'непримиримый' не найдено в модели.
Слово 'неонацизм' не найдено в модели.
Слово 'осложняться' не найдено в модели.
Слово 'восставить' не найдено в модели.
Слово 'весточка' не найдено в модели.
Слово 'орудийный' не найдено в модели.
Слово 'призывник' не найдено в модели.
Слово 'порхать' не найдено в модели.
Слово 'малозаметный' не найдено в модели.
Слово 'назойливый' не найдено в модели.
Слово 'изводить' не найдено в модели.
Слово 'жужжание' не найдено в модели.
Слово 'биотопический' не найдено в модели.
Слово 'инвайдер' не найдено в модели.
Слово 'почвообитать' не найдено в модели.
Слово 'мезофауна' не найдено в модели.
Слово 'верста' не найдено в модели.
Слово 'рго' не найдено в модели.
Слово 'сапроксильный' не найдено в модели.
Слово 'северцов' не найдено в модели.
Слово 'обследоваться' не найдено в модели.
Слово 'кассиля' не найдено в модели.
Слово 'гуашевый' не найдено в модели.
Слово 'знамённый' не найдено в модели.
Слово 'оправитьс

In [3]:
from gensim.models import KeyedVectors, Word2Vec
big = KeyedVectors.load_word2vec_format("ruscorpora_upos_skipgram_300_5_2018/model.vec", binary=False)

In [4]:
small = KeyedVectors.load_word2vec_format('vectors.txt', binary=False, no_header=True)

In [7]:
small_words = set(small.key_to_index.keys())
big_words = set(big.key_to_index.keys())

all_words = small_words.union(big_words)

new_vocab = {}
new_vectors = []

for word in all_words:
    if word in small_words and word in big_words:
        vec = (small.get_vector(word) + big.get_vector(word)) / 2
    elif word in small_words:
        vec = small.get_vector(word)
    else:
        vec = big.get_vector(word)

    new_vocab[word] = len(new_vectors)
    new_vectors.append(vec)

new_kv = KeyedVectors(vector_size=big.vector_size)
new_kv.add_vectors(list(new_vocab.keys()), new_vectors)

new_kv.save("glove.kv")